(FormulaeExample)=
# Example: Formulae file

An example a formulae file lives at `tests/Example_Config_Formulae/processing_formulae.csv`.

Here, this file will be read and the columns their expected values and use will be discussed. This is done using only the first row,
which takes the mean of the three thermocouples, having labels `DuctTC1`, `DuctTC2`, `DuctTC3`. These three thermocouples are the data obtained during acquisition, and correspond to the config file `Label`. Check out [config file example](NIConfigExample).

In [13]:
import polars as pl
import numpy as np
import matplotlib.pyplot as plt

formulae_file_path = "../tests/Example_Config_Formulae/processing_formulae.csv"
formulae_df = pl.read_csv(formulae_file_path)
formulae_df.columns = [i.strip() for i in formulae_df.columns]

with pl.Config() as cfg:
    cfg.set_tbl_cols(formulae_df.shape[1])
    cfg.set_tbl_rows(formulae_df.shape[0])
    cfg.set_fmt_str_lengths(100)
    cfg.set_float_precision(3)
    cfg.set_tbl_formatting("UTF8_FULL")
    # cfg.set_tbl_hide_column_data_types(True)
    cfg.set_tbl_hide_dataframe_shape(True)
    pl_cfg_json = cfg.save()
    print(formulae_df.head(5))

┌───────────┬─────────────────┬──────────────┬────────────────┬────────┬──────────┬────────────────┐
│ Label     ┆ RHS             ┆ Chart        ┆ Legend         ┆ Layout ┆ Position ┆ Processed_Unit │
│ ---       ┆ ---             ┆ ---          ┆ ---            ┆ ---    ┆ ---      ┆ ---            │
│ str       ┆ str             ┆ str          ┆ str            ┆ i64    ┆ i64      ┆ str            │
╞═══════════╪═════════════════╪══════════════╪════════════════╪════════╪══════════╪════════════════╡
│ Tmean     ┆ (DuctTC1 +      ┆  DuctTC      ┆  Mean gas temp ┆ 2      ┆ 2        ┆ K              │
│           ┆ DuctTC2 +       ┆              ┆                ┆        ┆          ┆                │
│           ┆ DuctTC3)/3      ┆              ┆                ┆        ┆          ┆                │
│           ┆ +273.15         ┆              ┆                ┆        ┆          ┆                │
├╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌

### Formulae file columns
1. `Label` : str
    - A unique label for each row.
    - This label indicates the left-hand-side of the equation to be processed.
    - A variable by this name will be created during post-processing.
3. `RHS` : str
    - The right-hand-side part of the equation.
    - Must be a string that can be executed in python. **For example, `DuctTC1**2` is `DuctTC1` raise to the exponent `2`. If you use `sqrt(DuctTC1)` will replace `sqrt` to `np.sqrt` to get the square root of `DuctTC1`. This is done in accordance to a dictionary that maps custom variable to numpy mathematical functions. See note below.**
    - Equation of the form `Label = RHS` is executed during post-processing.
    - See built-in python [exec](https://docs.python.org/3/library/functions.html#exec) function for additional details.
    - **Note: Can contain formulae that use numpy mathemetical functions on 1D array. A `Formulae_dict` (a dictionary) in {doc}`DAQUtils documentation<autoapi/firepydaq/utilities/DAQUtils/index>` indicates the currently acceptable variables that maps to a numpy function. You can add your custom `variable: function` map to this dictionary following the example given in the DAQUtils documentation.**
5. `Chart`: str
    - String indicating the chart where this channel's post processed data will be displayed.
    - This can be `None`, `Intermediate`, or `Constant` is you do not want to see this channel in the dashboard plots.
    - It is recommended to use `Constant` in the `Chart` column for row which is a constant used in post-processing. (Example: `KRosemont` and `KWeyer` labels in the above example formulae file.) 
    - It is recommended to use `Constant` in the `Chart` column for row which is a constant used in post-processing. (Example: `rho_e` and `ID` labels in the above example.
8. `Layout`: str/num
    - Same columns as that used in the config file
    - Must be in the form that can be converted into a natural number (1, 2, 3)
    - The type of layout corresponding to the `Chart` parameter.
    - All channels which are have the same `Chart` parameter must have the same `Layout` value.
    - This will generate a figure in the dashboard that will have \#`Layout` rows in vertical direction.
    - It is recommended to keep this value to a maximum of 3 for better visualization in the dashboard.
9. `Position`: str/num
    - Same columns as that used in the config file
    - Must be in the form that can be converted into a natural number (1, 2, 3)
    - Indicates the position in the `Chart` where this channel data will be displayed.
    - This value **must** be less than or equal to the `Layout` number.
10. `Processed_Unit`: str
    - Same columns as that used in the config file
    - The unit of the channel data once it is scaled according to AI and Scale minimum and maximum values.
    - This will be the y-axis label for the row.
11. `Legend`: str
    - Same columns as that used in the config file
    - Legend for the channel.
    - This will be displayed on the outside right of the plot in the dashboard.
    
**Please visit [Post processing example](PPExample) to see how the `Tmean` and other labels are processed.**